_Note: This is a SQL scoped notebook, and therefore should be attached to a Serverless SQL warehouse for execution._  

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');
DECLARE OR REPLACE VARIABLE resource_type STRING DEFAULT 'Encounter';
DECLARE OR REPLACE VARIABLE full_refresh BOOLEAN DEFAULT FALSE;

In [0]:
SET VARIABLE catalog_use = :catalog_use; 
SET VARIABLE schema_use = :schema_use;
SET VARIABLE resource_type = :resource_type; 
SET VARIABLE full_refresh = :full_refresh == 'true';

SELECT resource_type, full_refresh;

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
DECLARE OR REPLACE VARIABLE new_resource_keys BOOLEAN DEFAULT FALSE;

SET VARIABLE new_resource_keys = (
  WITH column_diff AS (
    SELECT column_name 
    FROM resource_schemas 
    WHERE 
      resourceType = resource_type
    EXCEPT
    SELECT column_name 
    FROM IDENTIFIER(catalog_use || ".information_schema.columns")
    WHERE 
      table_catalog = catalog_use
      AND table_schema = schema_use
      AND table_name = lower(resource_type)
      AND (column_name NOT LIKE 'bundle_uuid' AND column_name NOT LIKE resource_type)
  )
  SELECT count(*) > 0 FROM column_diff
);

SELECT new_resource_keys;

In [0]:
DECLARE OR REPLACE VARIABLE full_refresh_stmnt STRING; 

SET VARIABLE full_refresh_stmnt = (
  CASE when (full_refresh OR new_resource_keys) THEN "DROP TABLE IF EXISTS " || resource_type || ";" ELSE "SELECT 'Performing standard refresh of streaming table " || resource_type || "' AS message;" END
);

SELECT full_refresh_stmnt;

In [0]:
EXECUTE IMMEDIATE full_refresh_stmnt;

In [0]:
DECLARE OR REPLACE VARIABLE resource_keys STRING;

-- SET VARIABLE resource_keys = (
--   SELECT 
--     array_join(collect_list(DISTINCT column_name), "', '")
--   FROM 
--     resource_schemas
--   WHERE 
--     resourceType = resource_type
-- )

SET VARIABLE resource_keys = (
  WITH existing_colums as (
    SELECT 
      resource_type as resourceType
      ,column_name
      ,ordinal_position
    FROM 
      IDENTIFIER(catalog_use || ".information_schema.columns")
    WHERE 
      table_catalog = catalog_use
      AND table_schema = schema_use
      AND table_name = lower(resource_type)
    ORDER BY 
      ordinal_position
  )
  ,ordered_columns AS (
    SELECT
      t1.resourceType,
      t1.column_name,
      t2.ordinal_position
    FROM
      resource_schemas t1
      LEFT OUTER JOIN existing_colums t2 ON t1.resourceType = t2.resourceType AND t1.column_name = t2.column_name
    WHERE
      t1.resourceType = resource_type
    ORDER BY
      t2.ordinal_position
  ),
  ordered_values AS (
    SELECT
      collect_list(column_name) OVER (PARTITION BY resourceType ORDER BY ordinal_position) AS ordered_values,
      row_number() OVER (PARTITION BY resourceType ORDER BY ordinal_position DESC) AS row_num
    FROM
      ordered_columns
  )
  SELECT
    array_join(ordered_values, "', '")
  FROM
    ordered_values
  WHERE
    row_num = 1
);

select resource_keys;

In [0]:
DECLARE OR REPLACE VARIABLE resource_stmnt STRING;

SET VARIABLE resource_stmnt = "
CREATE OR REFRESH STREAMING TABLE " || resource_type || "
COMMENT 'Parsed FHIR " || resource_type || " Records'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  *
FROM (
  SELECT
    bundle_uuid
    ,fullUrl as "|| resource_type ||"_url
    ,key
    ,value
  FROM 
    STREAM(resources_exploded)
  WHERE 
    resourceType = '"|| resource_type ||"')
  PIVOT (
    first(value) FOR key IN ('" || resource_keys || "')
  );
";

SELECT resource_stmnt;


In [0]:
EXECUTE IMMEDIATE resource_stmnt;